In [ ]:
import pickle as pkl
import pandas as pd
import ast

In [ ]:
# Pull budget ascending auction data from each file and store in a dictionary
def pull_aa_data(file):
    return pd.read_csv(file)

def pull_fisher_data(file):
    return pd.read_csv(file)
# pathname = "results/toulouse_case_cap8_updated_20_ascending-auction-budgetbased_b-50.0_agentsall_dval40.0_outval1.0_pout1.0_freq500.0_pbound3000.0_receding_20250223_154045/results"
budgetbased_pathname = "results/toulouse_case_cap8_updated_20_ascending-auction-budgetbased_b-50.0_agentsall_dval40.0_outval1.0_pout1.0_freq500.0_pbound3000.0_receding_20250225_163015/results"
profitbased_pathname = "results/toulouse_case_cap8_updated_20_ascending-auction-profitbased_b-50.0_agentsall_dval40.0_outval1.0_pout1.0_freq500.0_pbound3000.0_receding_20250228_024433/results"
# budgetbased_pathname = "results/toulouse_case_cap7_updated_20_ascending-auction-budgetbased_b-50.0_agentsall_dval40.0_outval1.0_pout1.0_freq30.0_pbound3000.0_beta-method-none_alpha-1_receding_20250305_175621/results"
# profitbased_pathname = "results/toulouse_case_cap7_updated_20_ascending-auction-profitbased_b-50.0_agentsall_dval40.0_outval1.0_pout1.0_freq30.0_pbound3000.0_beta-method-none_alpha-1_receding_20250305_175208/results"

# budgetbased_pathname = "results/toulouse_case_cap7_updated_20_short_ascending-auction-budgetbased_b-50.0_agentsall_dval40.0_outval1.0_pout1.0_freq30.0_pbound3000.0_beta-method-none_alpha-1_receding_20250305_191500/results"
# profitbased_pathname = "results/toulouse_case_cap7_updated_20_short_ascending-auction-profitbased_b-50.0_agentsall_dval40.0_outval1.0_pout1.0_freq30.0_pbound3000.0_beta-method-none_alpha-1_receding_20250305_191550/results"

# Old one: profitbased_pathname = "results/toulouse_case_cap8_updated_20_ascending-auction-profitbased_b-50.0_agentsall_dval40.0_outval1.0_pout1.0_freq500.0_pbound3000.0_receding_20250225_130719/results"
# profitbased10_pathname = "results/toulouse_case_cap8_updated_20_ascending-auction-profitbased_b-50.0_agentsall_dval10.0_outval1.0_pout1.0_freq500.0_pbound3000.0_receding_20250225_133234/results"
# profitbased0_pathname =  "results/toulouse_case_cap8_updated_20_ascending-auction-profitbased_b-50.0_agentsall_dval0.0_outval1.0_pout1.0_freq500.0_pbound3000.0_receding_20250225_154950/results"
auction_ends = [20, 40, 60, 80, 100, 120, 140, 160, 180, 200, 220, 240, 260, 280, 300]
# auction_ends = [40]
budgetbased_files = [f"{budgetbased_pathname}/output_{end}.csv" for end in auction_ends]
profitbased40_files = [f"{profitbased_pathname}/output_{end}.csv" for end in auction_ends]
# profitbased10_files = [f"{profitbased10_pathname}/output_{end}.csv" for end in auction_ends]
# profitbased0_files = [f"{profitbased0_pathname}/output_{end}.csv" for end in auction_ends]
baa_data = {end: pull_aa_data(file) for end, file in zip(auction_ends, budgetbased_files)}
paa40_data = {end: pull_aa_data(file) for end, file in zip(auction_ends, profitbased40_files)}
# paa10_data = {end: pull_aa_data(file) for end, file in zip(auction_ends, profitbased10_files)}
# paa0_data = {end: pull_aa_data(file) for end, file in zip(auction_ends, profitbased0_files)}

In [ ]:
# Find average delay, number of rebases, average number of times rebased, etc.
def find_stats_baseline(df, rebased_agents=None):
    delays = [value for value in df['Delay'].tolist() if value >= 0]
    num_rebased_agents = len(df[df['Delay'] == -1]) + len(df[df['Delay'] == -2])
    # num_rebased_agents = 0
    # for agent in data:
    #     if agent["delay"] >= 0:
    #         delays.append(agent["delay"])
    #     elif agent["delay"] == -1 or agent["delay"] == -2:
    #         num_rebased_agents += 1
    return {
        "num_agents": df.shape[0],
        "delays": delays,
        "average_delay_of_allocated": sum(delays) / len(delays),
        "average_delay_of_delayed": sum(delays) / max(len([delay for delay in delays if delay > 0]), 1),
        "num_rebased_agents": num_rebased_agents
        }

def find_stats_fisher(df):
    def process_times(str):
        parsed = ast.literal_eval(str)
        return tuple(int(x) for x in parsed)
    df["Requested Time"] = df['"Requested Time (Arrival, Departure)"'].apply(process_times)
    df["Allocated Time"] = df['"Allocated Time (Arrival, Departure)"'].apply(process_times)
    df["Delay"] = df["Allocated Time"][0] - df["Requested_Time"][0]
    delays = [value for value in df['Delay'].tolist() if value >= 0]
    num_rebased_agents = len(df[df["Status"] == "rebased"])

    return {
        "num_agents": df.shape[0],
        "delays": df["Delay"].tolist(),
        "average_delay_of_allocated": sum(df["Delay"].tolist()) / max(len(df["Delay"].tolist()), 1),
    }

print(f"---------Budget Ascending Auction---------")
stats = {end: find_stats_baseline(data) for end, data in baa_data.items()}
for stat in stats.values():
    print(stat)

total_rebased = sum([stat['num_rebased_agents'] for stat in stats.values()])
total_num_agents = sum([stat['num_agents'] for stat in stats.values()])
print(f"Total rebased: {total_rebased}")
print(f"Percent rebased: {total_rebased / total_num_agents * 100}%")
total_delays = [len([val for val in stat['delays'] if val > 0]) for stat in stats.values()]
print(f"Total delayed: {sum(total_delays)}")
print(f"Average delay: {sum([sum(stat['delays']) for stat in stats.values()]) / sum(total_delays)}")

print(f"---------Profit Ascending Auction Drop Value=40---------")
profit40_stats = {end: find_stats_baseline(data) for end, data in paa40_data.items()}
for stat in profit40_stats.values():
    print(stat)

total_rebased = sum([stat['num_rebased_agents'] for stat in profit40_stats.values()])
total_num_agents = sum([stat['num_agents'] for stat in profit40_stats.values()])
print(f"Total rebased: {total_rebased}")
print(f"Percent rebased: {total_rebased / total_num_agents * 100}%")
total_delays = [len([val for val in stat['delays'] if val > 0]) for stat in profit40_stats.values()]
print(f"Total delayed: {sum(total_delays)}")
print(f"Average delay: {sum([sum(stat['delays']) for stat in profit40_stats.values()]) / sum(total_delays)}")

# print(f"---------Profit Ascending Auction Drop Value=10---------")
# profit10_stats = {end: find_stats_baseline(data) for end, data in paa10_data.items()}
# for stat in profit10_stats.values():
#     print(stat)

# total_rebased = sum([stat['num_rebased_agents'] for stat in profit10_stats.values()])
# total_num_agents = sum([stat['num_agents'] for stat in profit10_stats.values()])
# print(f"Total rebased: {total_rebased}")
# print(f"Percent rebased: {total_rebased / total_num_agents * 100}%")

# print(f"---------Profit Ascending Auction Drop Value=0---------")
# profit0_stats = {end: find_stats_baseline(data) for end, data in paa0_data.items()}
# for stat in profit0_stats.values():
#     print(stat)

# total_rebased = sum([stat['num_rebased_agents'] for stat in profit0_stats.values()])
# total_num_agents = sum([stat['num_agents'] for stat in profit0_stats.values()])
# print(f"Total rebased: {total_rebased}")
# print(f"Percent rebased: {total_rebased / total_num_agents * 100}%")



In [ ]:
fisher_data = {20: {'num_agents': 10, 'delays': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'average_delay_of_allocated': 0.0, 'average_delay_of_delayed': 0.0, 'num_rebased_agents': 0},
    40: {'num_agents': 14, 'delays': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1], 'average_delay_of_allocated': 0.14285714285714285, 'average_delay_of_delayed': 1.0, 'num_rebased_agents': 0},
    60: {'num_agents': 12, 'delays': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'average_delay_of_allocated': 0.0, 'average_delay_of_delayed': 0.0, 'num_rebased_agents': 0},
    80: {'num_agents': 3, 'delays': [0, 1, 0], 'average_delay_of_allocated': 0.3333333333333333, 'average_delay_of_delayed': 1.0, 'num_rebased_agents': 0},
    100: {'num_agents': 11, 'delays': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'average_delay_of_allocated': 0.0, 'average_delay_of_delayed': 0.0, 'num_rebased_agents': 0},
    120: {'num_agents': 14, 'delays': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'average_delay_of_allocated': 0.0, 'average_delay_of_delayed': 0.0, 'num_rebased_agents': 0},
    140: {'num_agents': 10, 'delays': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'average_delay_of_allocated': 0.0, 'average_delay_of_delayed': 0.0, 'num_rebased_agents': 0},
    160: {'num_agents': 12, 'delays': [0, 0, 0, 0, 0, 0, 0], 'average_delay_of_allocated': 0.0, 'average_delay_of_delayed': 0.0, 'num_rebased_agents': 5},
    180: {'num_agents': 20, 'delays': [0, 0, 0, 0, 1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'average_delay_of_allocated': 0.2222, 'average_delay_of_delayed': 1.33, 'num_rebased_agents': 2},
    200: {'num_agents': 16, 'delays': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'average_delay_of_allocated': 0.0, 'average_delay_of_delayed': 0.0, 'num_rebased_agents': 0},
    220: {'num_agents': 17, 'delays': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'average_delay_of_allocated': 0.0, 'average_delay_of_delayed': 0.0, 'num_rebased_agents': 2},
    240: {'num_agents': 14, 'delays': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], 'average_delay_of_allocated': 0.21428571428571427, 'average_delay_of_delayed': 1.5, 'num_rebased_agents': 0},
    260: {'num_agents': 20, 'delays': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'average_delay_of_allocated': 0.058823529411764705, 'average_delay_of_delayed': 1.0, 'num_rebased_agents': 3},
    280: {'num_agents': 3, 'delays': [0, 1, 0], 'average_delay_of_allocated': 0.3333333333333333, 'average_delay_of_delayed': 1.0, 'num_rebased_agents': 0}}

total_rebased = sum([stat['num_rebased_agents'] for stat in fisher_data.values()])
total_num_agents = sum([stat['num_agents'] for stat in fisher_data.values()])
print(f"---------Fisher Market---------")
print(f"Total rebased: {total_rebased}")
print(f"Percent rebased: {total_rebased / total_num_agents * 100}%")

In [ ]:
rebases_per_agent = {}
unallocated = 0
for end in auction_ends:
    # Grab data about the number of rebases per agent
    with open(f"{budgetbased_pathname}/budget_data_{end}.pkl", 'rb') as f:
        budgetbased_pkl_data = pkl.load(f)

    budgetbased_rebased_agents = budgetbased_pkl_data['rebased']
    # print(budgetbased_rebased_agents)
    for flight_id, flight in budgetbased_rebased_agents.items():
        if flight['rebase_count'] < 2:
            rebases_per_agent[flight_id] = flight['rebase_count'] + 1
        else:
            rebases_per_agent[flight_id] = 2
            unallocated += 1
print(rebases_per_agent)
print(f"Number of rebased agents: {len(rebases_per_agent)}")
print(f"Number of times rebased: {sum(rebases_per_agent.values())}")
print(f"Average number of rebases per agent: {sum(rebases_per_agent.values()) / len(rebases_per_agent)}")
print(f"Number of agents never allocated: {unallocated}")

rebases_per_agent = {}
unallocated = 0
for end in auction_ends:
    # Grab data about the number of rebases per agent
    with open(f"{profitbased_pathname}/profit_data_{end}.pkl", 'rb') as f:
        profitbased_pkl_data = pkl.load(f)

    profitbased_rebased_agents = profitbased_pkl_data['rebased']
    # print(budgetbased_rebased_agents)
    for flight_id, flight in profitbased_rebased_agents.items():
        if flight['rebase_count'] < 2:
            rebases_per_agent[flight_id] = flight['rebase_count'] + 1
        else:
            rebases_per_agent[flight_id] = 2
            unallocated += 1
print(rebases_per_agent)
print(f"Number of rebased agents: {len(rebases_per_agent)}")
print(f"Number of times rebased: {sum(rebases_per_agent.values())}")
print(f"Average number of rebases per agent: {sum(rebases_per_agent.values()) / len(rebases_per_agent)}")
print(f"Number of agents never allocated: {unallocated}")